In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools
from matplotlib.ticker import ScalarFormatter
pd.set_option('display.max_columns', None)

In [106]:
df_raw = pd.read_csv("dropout_rate.csv", sep=";", encoding="utf-8-sig")

In [107]:
df_raw

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,5,9,127.3,1,0,0,1,1,0,20,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,3,3,142.5,1,0,0,0,1,0,19,0,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,9,9,124.8,1,0,0,0,1,0,19,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,5,3,119.6,1,0,0,1,0,0,20,0,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,9,9,141.5,0,0,0,1,0,0,45,0,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,9773,1,1,125.0,1,1,1,5,4,122.2,0,0,0,1,1,0,19,0,0,6,7,5,13.600000,0,0,6,8,5,12.666667,0,15.5,2.8,-4.06,Graduate
4420,1,1,2,9773,1,1,120.0,105,1,1,9,9,119.0,1,0,1,0,0,0,18,1,0,6,6,6,12.000000,0,0,6,6,2,11.000000,0,11.1,0.6,2.02,Dropout
4421,1,1,1,9500,1,1,154.0,1,37,37,9,9,149.5,1,0,0,1,0,1,30,0,0,7,8,7,14.912500,0,0,8,9,1,13.500000,0,13.9,-0.3,0.79,Dropout
4422,1,1,1,9147,1,1,180.0,1,37,37,7,4,153.8,1,0,0,1,0,1,20,0,0,5,5,5,13.800000,0,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,Graduate


In [108]:
df_raw['Curricular grade Year'] = df_raw['Curricular units 1st sem (grade)'] + df_raw['Curricular units 2nd sem (grade)']
df_raw['Curricular units Year'] = df_raw['Curricular units 1st sem (approved)'] + df_raw['Curricular units 2nd sem (approved)']

df_raw['Curricular units failed Year'] = df_raw['Curricular units 1st sem (enrolled)'] + df_raw['Curricular units 2nd sem (enrolled)'] - df_raw['Curricular units Year']

In [109]:
df_raw.columns

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance\t', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd

### Chọn feature

In [110]:
ft_cols_qualitative = [
    'Tuition fees up to date',
    'Scholarship holder',
    'Debtor',
]

ft_cols_qualitative_to_one_hot = [
    
]

ft_cols_quantitative = [
    'Age at enrollment',
    'Curricular units failed Year',
    'Curricular grade Year',
    'Curricular units Year',
]

target_col = 'Target'

In [111]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_validate

def run_kfold_pipeline(
        df_raw: pd.DataFrame,
        ft_cols_qualitative: list[str],
        ft_cols_qualitative_to_one_hot: list[str],
        ft_cols_quantitative: list[str],
        target_col: str,
        scaler,
        model,
        n_splits: int = 5,
        random_state: int = 42,
        shuffle: bool = True,
        scoring=None
    ):
    
    binary_flag_cols = list(set(ft_cols_qualitative) - set(ft_cols_qualitative_to_one_hot))

    preprocessor = ColumnTransformer(
        transformers=[
            ("num",    scaler,                                   ft_cols_quantitative),
            ("onehot", OneHotEncoder(handle_unknown="ignore"),   ft_cols_qualitative_to_one_hot),
            ("binary", "passthrough",                            binary_flag_cols)
        ],
        remainder="drop"
    )

    pipe = Pipeline([
        ("prep", preprocessor),
        ("model", model)
    ])

    X = df_raw.drop(columns=[target_col])
    y = df_raw[target_col]

    cv = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=shuffle)
    results = cross_validate(pipe, X, y, cv=cv, n_jobs=-1,
                             return_train_score=True, scoring=scoring)

    return results



In [112]:
df = df_raw[ft_cols_qualitative + ft_cols_quantitative + [target_col]].copy()

df[target_col] = [1 if x == 'Dropout' else 0 for x in df[target_col]]
df

,Tuition fees up to date,Scholarship holder,Debtor,Age at enrollment,Curricular units failed Year,Curricular grade Year,Curricular units Year,Target
0,1,0,0,20,0,0.000000,0,1
1,0,0,0,19,0,27.666667,12,0
2,0,0,0,19,12,0.000000,0,1
3,1,0,0,20,1,25.828571,11,0
4,1,0,0,45,1,25.333333,11,0
...,...,...,...,...,...,...,...,...
4419,1,0,0,19,2,26.266667,10,0
4420,0,0,1,18,4,23.000000,8,1
4421,1,1,0,30,7,28.412500,8,1
4422,1,1,0,20,0,25.800000,10,0


### LogisticRegression

In [113]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler


cv_scores = run_kfold_pipeline(
    df_raw=df,
    ft_cols_qualitative=ft_cols_qualitative,
    ft_cols_qualitative_to_one_hot=ft_cols_qualitative_to_one_hot,
    ft_cols_quantitative=ft_cols_quantitative,
    target_col=target_col,
    scaler=MinMaxScaler(),
    model=LogisticRegression(max_iter=1000, C=0.1),
    scoring = {
        "acc": "accuracy",
        "recall": "recall",
        "precision": "precision",
        "f1": "f1",
        "roc_auc": "roc_auc"
    }
)

pd.DataFrame(cv_scores).describe(percentiles=[]).loc[["mean", "std", "min", "max"]]


,fit_time,score_time,test_acc,train_acc,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1,test_roc_auc,train_roc_auc
mean,0.008740,0.014800,0.863022,0.862738,0.659392,0.658164,0.886249,0.885023,0.755775,0.754913,0.909646,0.910143
std,0.000751,0.000996,0.009670,0.003546,0.013108,0.006710,0.037371,0.006812,0.013630,0.006581,0.006702,0.001861
min,0.008034,0.013543,0.848588,0.859887,0.644366,0.650836,0.837838,0.877503,0.735178,0.749747,0.899611,0.908622
max,0.009878,0.016138,0.871041,0.867759,0.679577,0.666667,0.933673,0.894923,0.770459,0.764113,0.914693,0.912576


### RandomForestClassifier

In [118]:
from sklearn.ensemble import RandomForestClassifier

cv_scores = run_kfold_pipeline(
    df_raw=df,
    ft_cols_qualitative=ft_cols_qualitative,
    ft_cols_qualitative_to_one_hot=ft_cols_qualitative_to_one_hot,
    ft_cols_quantitative=ft_cols_quantitative,
    target_col=target_col,
    scaler=MinMaxScaler(),
    model=RandomForestClassifier(max_depth=10,
        n_estimators=100,
        min_samples_leaf=20,
    ),
    scoring = {
        "acc": "accuracy",
        "recall": "recall",
        "precision": "precision",
        "f1": "f1",
        "roc_auc": "roc_auc"
    }
)

pd.DataFrame(cv_scores).describe(percentiles=[]).loc[["mean", "std", "min", "max"]]

,fit_time,score_time,test_acc,train_acc,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1,test_roc_auc,train_roc_auc
mean,0.163171,0.022798,0.873194,0.879577,0.730484,0.742258,0.853850,0.863699,0.787269,0.798373,0.913126,0.930437
std,0.006373,0.000839,0.008161,0.001376,0.014747,0.003669,0.018309,0.005048,0.013292,0.002082,0.007302,0.001491
min,0.158304,0.021993,0.863277,0.877401,0.712281,0.739437,0.824701,0.858443,0.773832,0.794896,0.901947,0.929322
max,0.174313,0.024000,0.883484,0.880757,0.750000,0.746702,0.869388,0.869700,0.805293,0.800378,0.920432,0.932958


In [116]:
from xgboost import XGBClassifier

cv_scores = run_kfold_pipeline(
    df_raw=df,
    ft_cols_qualitative=ft_cols_qualitative,
    ft_cols_qualitative_to_one_hot=ft_cols_qualitative_to_one_hot,
    ft_cols_quantitative=ft_cols_quantitative,
    target_col=target_col,
    scaler=MinMaxScaler(),
    model=XGBClassifier(
        max_depth=3,
        learning_rate=0.1,
        n_estimators=100,
        reg_lambda=1,
        reg_alpha=1,
    ),
    scoring = {
        "acc": "accuracy",
        "recall": "recall",
        "precision": "precision",
        "f1": "f1",
        "roc_auc": "roc_auc"
    }
)

pd.DataFrame(cv_scores).describe(percentiles=[]).loc[["mean", "std", "min", "max"]]

,fit_time,score_time,test_acc,train_acc,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1,test_roc_auc,train_roc_auc
mean,0.031827,0.012953,0.875002,0.884098,0.736810,0.750000,0.854427,0.871261,0.791106,0.806090,0.914573,0.931819
std,0.001256,0.000455,0.005402,0.001652,0.012155,0.002926,0.018469,0.004386,0.007913,0.002647,0.005577,0.001768
min,0.030101,0.012737,0.870056,0.881638,0.721831,0.747359,0.835938,0.865580,0.784240,0.802265,0.906711,0.929819
max,0.033273,0.013766,0.883484,0.886126,0.753521,0.754617,0.875519,0.877572,0.803810,0.808914,0.919836,0.933805
